# Revisión de datos de los archivos csv de Open Contracting Partnership. Data Set del 2018.

Usando el buscador de OpenContracting se buscaron datos sobre Chile (buscador: https://data.open-contracting.org/es/search/). Se decidió descargarlos archivos de los años 2018 y 2021, para tener una muestra de datos que contenga información sobre licitaciones en un rango de tiempo cercano a estas fechas.

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
from os import path
import pandas as pd

lista_doc = ["main.csv","awards.csv", "awards_items.csv", "awards_suppliers.csv"]
contador = 0
for documento in lista_doc:
    print(documento)
    camino = path.join("Chile","2021",documento)
    if contador == 0:
        df_main = pd.read_csv(camino, delimiter=",")
        
    elif contador == 1:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas = list(df_e.columns.values)
        
        if "description" in lista_columnas :
            df_e.rename(columns={'description': f'description_{documento}'}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
        df_main = df_main.merge(df_e, left_on="_link", right_on="_link_main")
        
    else:
        df_e = pd.read_csv(camino, delimiter=",")
        lista_columnas =  list(df_e.columns.values)
        if "description" in lista_columnas:
            df_e.rename(columns={'id': f'id_{documento}', "_link": f"_link_{documento}"}, inplace=True)
            
        df_e.rename(columns={'id': f'id_{documento}'}, inplace=True)
        df_main = df_main.merge(df_e, left_on="_link_awards.csv", right_on="_link_awards")
    contador += 1
    
df_main.info()

Se cargan los documentos main, awards, award_items y award_suppliers, a un DataFrame, se limpiaran los datos haciendo drop de columnas irrelevantes y/o repetidas.

In [ ]:
df_2021 = df_main

In [ ]:
df_2021.drop(columns=["id","ocid","tender_procuringEntity_id", "_link_awards_items.csv","id_awards_items.csv", "tender_id", "buyer_id", "_link_awards_x", "_link_awards_y"], inplace=True)
df_2021.drop(columns=["classification_uri", "_link_awards.csv", "id_awards.csv", "id_awards_suppliers.csv", "unit_value_unitOfAccount", "value_unitOfAccount", "language" ], inplace=True)
df_2021.columns

Se eliminan las columnas con datos que permiten la identificación dentro del sistema del que se originan, puesto que no son pertinentes para los analisis a realizarse.

In [ ]:
df_2021['date_x'] = pd.to_datetime(df_2021["date_x"])

In [ ]:
df_2021['date_y'] = pd.to_datetime(df_2021['date_y'])

In [ ]:
df_2021['date_y']

In [ ]:
df_2021[["_link_y", "_link_main_y", "_link_main_x", "_link_x"]]

Se revisa el contenido de las columnas "_link_main", "_link_y", "_link_main_y", "_link_main_x", "_link_x y se observa que _link_main y las columnas derivadas de este nombre son identicas entre ellas y con la columna _link_x, se eliminaran todas excepto main, ademas la columna _link_y no es relevancia para el DataFrame, puesto que es identica a la columna _link_main.

In [ ]:
df_2021.drop(columns=["_link_y", "_link_main_y", "_link_main_x", "_link_main", "tag","classification_id", "classification_scheme", "tender_title", "description", "tender_status", "status", "initiationType", "tender_value_unitOfAccount", "description_awards.csv", "title", "tender_description"], inplace=True)
df_main.head()
df_2021.info()

In [ ]:
df_2021[["date_y", "date_x"]]

Se comparan _date_y y _date_x y se observa que son distintas por lo que se mantienen, pero se cambia el nombre.

In [ ]:
df_2021.rename(columns={"name": "Award_supplier_name", 'date_y': 'date_award', "date_x": 'date_main'}, inplace= True)

Se revisarán que las filas no posean solo un dato, si sucede, se eliminarán. (Se ignorarán las filas que evidentemente poseen más de un dato).

In [ ]:
for columna in df_2021.columns:
    print(columna, df_2021[columna].unique())

In [ ]:
df_iguales = pd.DataFrame(df_2021["tender_value_amount"] == df_2021["value_amount"])
df_iguales.head()

In [ ]:
df_iguales.rename(columns={0: "tender_value_amount == value_amount"}, inplace=True)

In [ ]:
df_iguales["tender_value_amount == unit_value_amount"] = df_2021["tender_value_amount"] == df_2021["unit_value_amount"]
df_iguales["value_amount == unit_value_amount"] = df_2021["value_amount"] == df_2021["unit_value_amount"]

df_iguales["tender_value_currency == value_currency"] = df_2021["tender_value_currency"] == df_2021["value_currency"]
df_iguales["tender_value_currency == unit_value_currency"] = df_2021["tender_value_currency"] == df_2021["unit_value_currency"]
df_iguales["unit_value_currency == value_currency"] = df_2021["unit_value_currency"] == df_2021["value_currency"]
df_iguales["buyer == procuring_entity"] = df_2021["buyer_name"] == df_2021["tender_procuringEntity_name"]

In [ ]:
for columna in df_iguales.columns:
    print(columna, df_iguales[columna].unique())

Se observa que solo las columnas buyer y procuring_entity son iguales entre si, por lo que se eliminará buyer_name.

In [ ]:
duplicates = df_2021.duplicated(keep=False)
dups = df_2021[duplicates]
dups.head(10)

Volviendo a revisar tenemos que no hay dato NaN en los datos de 2021, pero si hay datos duplicados

In [ ]:
df_2021.dropna()

In [ ]:
df_2021 = df_2021.drop_duplicates()

In [ ]:
df_2021.drop_duplicates()

In [ ]:
df_2021.duplicated().unique()

Con esto corroboramos de que efectivamente no existe fila repetida alguna

In [ ]:
columnas_nan_full = df_2021.columns[df_2021.isna().all()]
columnas_nan_full

In [ ]:
df_2021 = df_2021[['date_main', 'tender_procurementMethodDetails',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_name',
       'tender_value_amount', 'tender_value_currency', 'buyer_name',
       'planning_budget_id', 'planning_budget_description',
       'planning_budget_amount_amount', 'planning_budget_amount_currency', 'date_award', 'value_amount',
       'value_currency', 'quantity', 'unit_name', 'unit_value_amount',
       'unit_value_currency', '_link_main', 'Award_supplier_name']]

Verificamos que todo este correcto

In [ ]:
verify = df_2021.columns[df_2021.isna().all()]
verify

Ahora buscaremos tener solo las filas con las que quedamos del notebook anterior (2018)

In [ ]:
df_2021 = df_2021[['date_main', 'buyer_name', 'tender_procurementMethodDetails',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_name',
       'tender_value_amount', 'tender_value_currency', 'date_award',
       'value_amount', 'value_currency', 'quantity',
       'unit_name', 'unit_value_amount', 'unit_value_currency', '_link_main',
       'Award_supplier_name']]

In [ ]:
df_2021.info()

## Camino a eliminar/reemplazar los datos NaN

In [ ]:

tipos_licitaciones ={'Licitación Pública Menor a 100 UTM (L1)': "L1",
       'Licitación Pública Entre 100 y 1000 UTM (LE)': "LE",
       'Licitación Pública Mayor 1000 UTM (LP)': "LP",
       'Licitación Pública entre a 2000 y 5000 UTM (LQ)':"LQ",
       'Licitación Pública Mayor a 5000 (LR)' :"LR",
       'Licitación Privada entre a 2000 y 5000 UTM (H2)':"H2",
       'Licitación Pública Servicios personales especializados (LS)':"LS",
       "Licitación Privada por Licitación Pública anterior sin oferentes (A1)" : "A1",
       "Licitación Privada por Remanente de Contrato anterior (B1)": "B1",
       "Licitación Privada por Convenios con Personas Jurídicas Extranjeras fuera del Territorio Nacional (E1)": "E1",
       "Licitación Privada por Servicios de Naturaleza Confidencial (F1)": "F1",
       "Licitación Privada por otras causales, excluidas de la ley de Compras (J1)": "J1",
       "Licitación Privada entre 100 y 1000 UTM" :"CO", "Licitación Privada Mayor a 1000 UTM": "B2",
       "Trato Directo por Producto de Licitación Privada anterior sin oferentes o desierta": "A2",
       "Trato Directo por Proveedor Único (D1)": "D1","Licitación Privada Menor a 100 UTM" : "E2", "Trato Directo (Cotización) (C2)" : "C2",
       "Compra Directa (Orden de compra) (C1)" : "C1", "Trato Directo (Cotización) (F2)" :"F2", "Compra Directa (Orden de compra) (F3)" : "F3",
       "Directo (Cotización) (G2)" : "G2", "Compra Directa (Orden de compra) (G1)" : "G1",
       "Orden de Compra menor a 3 UTM (R1)" : "R1", "Orden de Compra sin Resolución (CA)" : "CA", 
       "Orden de Compra proveniente de adquisición sin emisión automática de OC (SE)" : "SE"}


df_2021["tender_procurementMethodDetails"] = df_2021["tender_procurementMethodDetails"].map(tipos_licitaciones)
df_2021["tender_procurementMethodDetails"] = df_2021["tender_procurementMethodDetails"].astype("category")

In [ ]:
df_2021["unit_value_currency"].isna().unique()

In [ ]:
df_2021["date_main"] = pd.to_datetime(df_2021['date_main'])
df_2021["date_award"] = pd.to_datetime(df_2021['date_award'])
df_2021["tender_enquiryPeriod_startDate"] = pd.to_datetime(df_2021['tender_enquiryPeriod_startDate'])
df_2021["tender_enquiryPeriod_endDate"] = pd.to_datetime(df_2021['tender_enquiryPeriod_endDate'])
df_2021['tender_tenderPeriod_startDate'] = pd.to_datetime(df_2021['tender_tenderPeriod_startDate'])
df_2021["tender_tenderPeriod_endDate"] = pd.to_datetime(df_2021['tender_tenderPeriod_endDate'])
df_2021['tender_awardPeriod_startDate'] = pd.to_datetime(df_2021['tender_awardPeriod_startDate'])

Notamos un error grave en la fila '703633', cual que tiene una fecha en 'tender_awardPeriod_endDate' de 2999-03-15T12:17:00Z. Al no poder 'inventarnos' alguna flecha y reemplazar ese valor optaremos por eliminarla

In [ ]:
df_2021.drop(df_2021[df_2021["tender_awardPeriod_endDate"] == "2999-03-15T12:17:00Z"].index, axis=0, inplace=True)

In [ ]:
df_2021['tender_awardPeriod_endDate'] = pd.to_datetime(df_2021['tender_awardPeriod_endDate'])
df_2021['tender_enquiryPeriod_durationInDays'] = df_2021['tender_enquiryPeriod_endDate'] - df_2021['tender_enquiryPeriod_startDate'] 
df_2021['tender_awardPeriod_durationInDays'] = df_2021['tender_awardPeriod_endDate'] - df_2021['tender_awardPeriod_startDate']
df_2021['tender_tenderPeriod_durationInDays'] = df_2021['tender_tenderPeriod_endDate'] - df_2021['tender_tenderPeriod_startDate']

# Filtrado por fecha
Así como en el notebook del 2018, filtraremos las fechas, de forma que queden solo entre (2020-09-01) y (2021-12-31).

In [ ]:
fecha_lim_menor = '2020-09-01'
df_filtrado = df_2021.loc[df_2021['tender_tenderPeriod_startDate'] >= fecha_lim_menor]

fecha_lim_mayor = '2021-12-31'
df_filtrado = df_filtrado.loc[df_filtrado['tender_awardPeriod_endDate'] <= fecha_lim_mayor]

In [ ]:
df_filtrado

In [ ]:
df_filtrado.reset_index(drop=True, inplace=True)

# Filtrado respecto a NaN
Aplicaremos la misma logica vista en 2018 para eliminar todos los datos POSIBLES NaN. Remarcamos posibles debido a que dependiendo del contexto podemos igualmente usarlos y reemplazarlos con un "No aplica"

In [ ]:
Datos_with_NaN = df_filtrado.loc[:, df_filtrado.isna().any()]
print(f"La cantidad de datos NaN es de: {len(Datos_with_NaN[Datos_with_NaN.isna().any(axis=1)])} en todo el dataframe de {len(df_filtrado)}")

In [ ]:
nan_tender_v = df_filtrado[df_filtrado['value_currency'].isna()]
nan_tender_v[["value_currency", "value_amount","unit_value_currency", "unit_value_amount","tender_value_amount", "tender_value_currency", "tender_hasEnquiries"]]

In [ ]:
df_filtrado["value_currency"] = df_filtrado["value_currency"].replace(np.nan, "No aplica")
df_filtrado["unit_value_currency"] = df_filtrado["unit_value_currency"].replace(np.nan, "No aplica")
df_filtrado["unit_value_amount"] = df_filtrado["unit_value_amount"].replace(np.nan, "No aplica")
df_filtrado["value_amount"] = df_filtrado["value_amount"].replace(np.nan, "No aplica")
df_filtrado["tender_value_currency"] = df_filtrado["tender_value_currency"].replace(np.nan, "No aplica")
df_filtrado["tender_value_amount"] = df_filtrado["tender_value_amount"].replace(np.nan, "No aplica")

In [ ]:
df_filtrado[["tender_tenderPeriod_durationInDays", "tender_awardPeriod_durationInDays", "tender_enquiryPeriod_durationInDays"]].info()

In [ ]:
df_filtrado.info()

In [ ]:
df_2021 = df_filtrado

In [ ]:
df_ver = df_2021[['buyer_name', 'tender_procuringEntity_name']]

In [ ]:
df_2021['buyer_name'] = df_2021['tender_procuringEntity_name']

In [ ]:
df_2021 = df_2021[['date_main', 'buyer_name', 'tender_procurementMethodDetails',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays',
       'tender_value_amount', 'tender_value_currency', 'date_award',
       'value_amount', 'value_currency', 'quantity', 'unit_name',
       'unit_value_amount', 'unit_value_currency', '_link_main',
       'Award_supplier_name']]

In [ ]:
df_2021.info()